<a href="https://colab.research.google.com/github/Itsuki-Hamano123/experiment_search_engine/blob/master/elmo_search_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import datetime
from pprint import pprint
import pickle
import time
import numpy as np
import pandas as pd

## 形態素解析ツール用意
https://qiita.com/Fulltea/items/90f6ebe6dcceaf64eaef

In [2]:
# mecabのインストール
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.5)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.8)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.3)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already install

In [3]:
# Neologd辞書のインストール
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

fatal: destination path 'mecab-ipadic-neologd' already exists and is not an empty directory.
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-NEologd] :     diff => ok
[install-mecab-ipadic-NEologd] :     tar => ok
[install-mecab-ipadic-NEologd] :     unxz => ok
[install-mecab-ipadic-NEologd] :     xargs => ok
[install-mecab-ipadic-NEologd] :     grep => ok
[install-mecab-ipadic-NEo

In [0]:
import MeCab
import subprocess

In [5]:
!ls

drive  mecab-ipadic-neologd  sample_data


In [6]:
cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
neologd_path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
# Neologd辞書の設定
m_neologd = MeCab.Tagger("-d {0}".format(neologd_path))

m_neologd_sample_txt = "8月3日に放送された「中居正広の金曜日のスマイルたちへ」(TBS系)で、1日たった5分でぽっこりおなかを解消するというダイエット方法を紹介。キンタロー。のダイエットにも密着。"

print("Mecab ipadic NEologd:\n",m_neologd.parse(m_neologd_sample_txt))

Mecab ipadic NEologd:
 8月3日	名詞,固有名詞,一般,*,*,*,8月3日,ハチガツミッカ,ハチガツミッカ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
放送	名詞,サ変接続,*,*,*,*,放送,ホウソウ,ホーソー
さ	動詞,自立,*,*,サ変・スル,未然レル接続,する,サ,サ
れ	動詞,接尾,*,*,一段,連用形,れる,レ,レ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
「	記号,括弧開,*,*,*,*,「,「,「
中居正広の金曜日のスマイルたちへ	名詞,固有名詞,一般,*,*,*,中居正広の金曜日のスマイルたちへ,ナカイマサヒロノキンヨウビノスマイルタチヘ,ナカイマサヒロノキンヨービノスマイルタチヘ
」(	記号,一般,*,*,*,*,*
TBS	名詞,固有名詞,一般,*,*,*,TBS,ティービーエス,ティービーエス
系	名詞,接尾,一般,*,*,*,系,ケイ,ケイ
)	記号,一般,*,*,*,*,*
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
、	記号,読点,*,*,*,*,、,、,、
1日	名詞,固有名詞,一般,*,*,*,1日,ツイタチ,ツイタチ
たった	副詞,助詞類接続,*,*,*,*,たった,タッタ,タッタ
5分	名詞,固有名詞,一般,*,*,*,5分,ゴフン,ゴフン
で	助詞,格助詞,一般,*,*,*,で,デ,デ
ぽっこりおなか	名詞,固有名詞,一般,*,*,*,ぽっこりおなか,ポッコリオナカ,ポッコリオナカ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
解消	名詞,サ変接続,*,*,*,*,解消,カイショウ,カイショー
する	動詞,自立,*,*,サ変・スル,基本形,する,スル,スル
という	助詞,格助詞,連語,*,*,*,という,トイウ,トユウ
ダイエット方法	名詞,固有名詞,一般,*,*,*,ダイエット方法,ダイエットホウホウ,ダイエットホウホー
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
紹介	名詞,サ変接続,*,*,*,*,紹介,ショウカイ,ショーカイ
。	記号,句点,*,*,*,*,。,。,。
キンタロー。	名詞,固有名詞,一般,*,*,*,キンタロー。,キンタロー,キンタロー
の	助詞,連体化,*,*,*,*,の,ノ,ノ
ダイエット	名詞,サ変接続,*,*,*,*,ダイエット,ダイエッ

## ELMoモデルの準備
ELMoForManyLangsを使う

In [7]:
# ELMoForManyLangsに必要なライブラリ
!pip install allennlp

### ELMoForManyLangsを利用
https://github.com/HIT-SCIR/ELMoForManyLangs#use-elmoformanylangs-programmatically

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
cd drive/My Drive/機械学習練習/全文検索エンジン作成

/content/drive/My Drive/機械学習練習/全文検索エンジン作成


In [10]:
!ls

build			   exports
configs			   faiss-gpu-1.4.0-py36_cuda9.2.148_1.tar.bz2
datasets		   faiss-gpu-1.4.0-py36_cuda9.2.148_1.tar.bz2.1
dist			   info
elmoformanylangs	   lib
elmoformanylangs.egg-info  mecab-ipadic-neologd
elmo_search_engine.ipynb   README.md
embedding		   setup.py


In [11]:
# ELMoForManyLangsのセットアップ
!python setup.py install

running install
running bdist_egg
running egg_info
writing elmoformanylangs.egg-info/PKG-INFO
writing dependency_links to elmoformanylangs.egg-info/dependency_links.txt
writing requirements to elmoformanylangs.egg-info/requires.txt
writing top-level names to elmoformanylangs.egg-info/top_level.txt
reading manifest file 'elmoformanylangs.egg-info/SOURCES.txt'
writing manifest file 'elmoformanylangs.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/elmoformanylangs
copying build/lib/elmoformanylangs/frontend.py -> build/bdist.linux-x86_64/egg/elmoformanylangs
copying build/lib/elmoformanylangs/__init__.py -> build/bdist.linux-x86_64/egg/elmoformanylangs
copying build/lib/elmoformanylangs/elmo.py -> build/bdist.linux-x86_64/egg/elmoformanylangs
copying build/lib/elmoformanylangs/utils.py -> build/bdist.linux-x86_64/egg/elmoformanylangs
copying build/

In [0]:
from elmoformanylangs import Embedder

In [0]:
def time_watch(func):
    """時間計測用デコレータ関数
    
    Parameters
    -----
    func : function
        実行したい関数

    """
    import functools
    import datetime
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = datetime.datetime.today()
        result = func(*args, **kwargs)
        end = datetime.datetime.today()
        print("実行時間:{time}".format(time=end - start))
        return result
    return wrapper

### 日本語の事前学習済み重みのダウンロード
embeddingディレクトに格納した<br>
https://qiita.com/mkt3/items/9577b63900109ff91665


In [0]:
# モデルのディレクトリ
MODEL_DIR = './embedding/word_unit_embedding_model/'

In [15]:
elmo_model = Embedder(MODEL_DIR, batch_size=30)

2020-04-04 15:30:34,820 INFO: word embedding size: 387179
2020-04-04 15:30:42,925 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(387179, 100, padding_idx=3)
    )
    (projection): Linear(in_features=100, out_features=512, bias=True)
  )
  (encoder): ElmobiLm(
    (forward_layer_0): LstmCellWithProjection(
      (input_linearity): Linear(in_features=512, out_features=16384, bias=False)
      (state_linearity): Linear(in_features=512, out_features=16384, bias=True)
      (state_projection): Linear(in_features=4096, out_features=512, bias=False)
    )
    (backward_layer_0): LstmCellWithProjection(
      (input_linearity): Linear(in_features=512, out_features=16384, bias=False)
      (state_linearity): Linear(in_features=512, out_features=16384, bias=True)
      (state_projection): Linear(in_features=4096, out_features=512, bias=False)
    )
    (forward_layer_1): LstmCellWithProjection(
      (input_linearity): Line

### 入力文書の前処理を定義

In [0]:
def is_stop_word(pos):
    """
    品詞からストップワードか判断

    Parameters
    -----
    pos : string
        品詞

    Returns
    -----
    stop_word_flag : boolean
    """
    stop_word_list = ["助詞", "助動詞", "記号", "BOS/EOS"]
    stop_word_flag = True
    if pos in stop_word_list:
        stop_word_flag = False
    return stop_word_flag

def split_text(text):
    """
    文章をわかち書きする

    Parameters
    -----
    text : string

    Returns
    -----
    result : list of string
    """
    result = []
    node = m_neologd.parseToNode(text)
    while node:
        if is_stop_word(node.feature.split(",")[0]):
            result.append(node.surface)
        node = node.next
    return result

In [0]:
def preprocessing_documents(documents):
    """
    文書をELMo用に前処理

    Parameters
    -----
    documents : list of string

    Returns
    -----
    results : list of list of string
    """
    results = []
    # バグ対処
    m_neologd.parse('')
    for document in documents:
        result = split_text(document)
        results.append(result)
    return results

In [0]:
elmo_sample_docments = ["私は犬", "犬は私", "今日の天気は雪のち晴れです"]

In [19]:
proc_documents = preprocessing_documents(elmo_sample_docments)
proc_documents

[['私', '犬'], ['犬', '私'], ['今日の天気', '雪', 'のち', '晴れ']]

### ELMoで単語の分散表現を予測

In [0]:
@time_watch
def predict_word_vec(model, proc_documents):
    """
    モデルを使い単語の分散表現を予測

    Parmeters
    -----
    model : model_obj
    proc_documents : proc_documents_obj
    
    Returns
    -----
    pred_vector : list of nparray
    """
    pred_vector = model.sents2elmo(proc_documents)
    return pred_vector

In [21]:
documnets_words_vec = predict_word_vec(elmo_model, proc_documents)

2020-04-04 15:30:45,133 INFO: 1 batches, avg len: 4.7


実行時間:0:00:00.038836


In [22]:
print(documnets_words_vec[0].shape)
print(type(documnets_words_vec[0]))
documnets_words_vec

(2, 1024)
<class 'numpy.ndarray'>


[array([[-0.3822073 , -0.39298368, -0.19943719, ..., -0.0947518 ,
          0.38196918, -0.06013573],
        [-0.99855757, -0.7653184 , -0.67188597, ...,  0.38484803,
         -0.14899619, -0.22106065]], dtype=float32),
 array([[-0.85666806, -0.24317439, -0.31542605, ..., -0.16575427,
          0.06315442, -0.31759426],
        [-0.29320082, -0.47321966, -0.83421296, ..., -0.40913573,
          0.21732388,  0.24414034]], dtype=float32),
 array([[ 0.10117573, -0.23624979,  0.00397639, ..., -0.2328723 ,
          0.17794509,  0.0607481 ],
        [-0.90098685, -0.80577874, -0.20233335, ...,  0.13288653,
          0.4778336 , -0.45927873],
        [ 1.2292985 ,  0.98217875, -0.351058  , ...,  0.05930225,
          0.5154347 , -0.4115046 ],
        [-0.65625954,  1.4385008 , -0.17021495, ..., -0.32545912,
          0.10576967,  0.18586273]], dtype=float32)]

#### 実行結果
「私は犬」と「犬は私」の２つの文章において、それぞれの文章で「私」「犬」は文章内での順番が考慮され、別々の分散表現が得られていることが分かる。

### 単語ベクトルから文書ベクトルを算出
今回は簡易的に、単語ベクトルの合計値を文書ベクトルする<br>

In [0]:
@time_watch
def calc_doc_vec(documnets_words_vec):
    """
    単語ベクトルから文書ベクトルを計算

    Parameters
    -----
    documents_word_vec : list of nparray

    Returns
    -----
    documents_vec : ndarray(文書数, 1つの文書内の次元数)

    Notes
    -----
    文書ベクトルの算出方法 : 文書内に含まれる全単語ベクトルの平均値
    """
    documnets_vec = []
    for document_words_vec in documnets_words_vec:
        document_vec = np.sum(document_words_vec, axis=0) / document_words_vec.shape[0]
        documnets_vec.append(document_vec)

    documnets_vec = np.array(documnets_vec)
    return documnets_vec

In [24]:
documnets_vec = calc_doc_vec(documnets_words_vec=documnets_words_vec)
documnets_vec

実行時間:0:00:00.000258


array([[-0.6903824 , -0.57915103, -0.43566158, ...,  0.14504811,
         0.1164865 , -0.1405982 ],
       [-0.5749344 , -0.35819703, -0.5748195 , ..., -0.287445  ,
         0.14023915, -0.03672696],
       [-0.05669305,  0.34466273, -0.17990749, ..., -0.09153567,
         0.31924576, -0.15604313]], dtype=float32)

In [25]:
print(documnets_vec.shape)
documnets_vec

(3, 1024)


array([[-0.6903824 , -0.57915103, -0.43566158, ...,  0.14504811,
         0.1164865 , -0.1405982 ],
       [-0.5749344 , -0.35819703, -0.5748195 , ..., -0.287445  ,
         0.14023915, -0.03672696],
       [-0.05669305,  0.34466273, -0.17990749, ..., -0.09153567,
         0.31924576, -0.15604313]], dtype=float32)

## faissのインストール
参考：https://qiita.com/SakuraDk/items/0c03b7ebd4aef8d6caed#colaboratory%E4%B8%8A%E3%81%A7%E3%82%A4%E3%83%B3%E3%82%B9%E3%83%88%E3%83%BC%E3%83%AB

In [26]:
# ダウンロードは一度のみ実行で大丈夫
!wget https://anaconda.org/pytorch/faiss-gpu/1.4.0/download/linux-64/faiss-gpu-1.4.0-py36_cuda9.2.148_1.tar.bz2
!tar xvjf faiss-gpu-1.4.0-py36_cuda9.2.148_1.tar.bz2

--2020-04-04 15:30:46--  https://anaconda.org/pytorch/faiss-gpu/1.4.0/download/linux-64/faiss-gpu-1.4.0-py36_cuda9.2.148_1.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.93.24, 104.17.92.24, 2606:4700::6811:5d18, ...
Connecting to anaconda.org (anaconda.org)|104.17.93.24|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5a15c9cc393ec612061f033a/5b92fa28ab12f91341118b86?response-content-disposition=attachment%3B%20filename%3D%22faiss-gpu-1.4.0-py36_cuda9.2.148_1.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27faiss-gpu-1.4.0-py36_cuda9.2.148_1.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=60&X-Amz-Date=20200404T153046Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEG0aCXVzLWVhc3QtMSJHMEUCIAGbsM2zofCuG5xALCLt%2B1sI7x0fqiZgSBk0U39z6HjnAiEAxXci1RkhzuLO5yLA9JqWxVJgV5cAOgvXysOfWj71IQAqtAMIdhAAGgw0NTU4NjQwOTgzNzgiDCHOqUKaxxGkvMSMPCqRA9wphSJU8Jl6z

In [27]:
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

In [0]:
import faiss

## 検索エンジン作成

### 検索対象となるドキュメント読み込み

In [29]:
DATA_DIR = "./datasets"
DATA_PATH = "a_ntcir_all_data.csv"
TEXT_COLUMN = "text"

headers = [TEXT_COLUMN]
register_documents = pd.read_csv(DATA_DIR+"/"+DATA_PATH, names=headers)
print("データ件数:{size}".format(size=len(register_documents)))
register_documents.index = register_documents.index + 1
register_documents.head()

データ件数:4125


,text
1,日本選手としては村上雅則（ジャイアンツ）、野茂英雄（ドジャース）、鈴木誠（マリナーズ）、長谷...
2,米大リーグのニューヨーク・メッツは１３日、フリーエージェント（ＦＡ）宣言した前ヤクルトの吉井...
3,竹下氏流の「広く浅く」という資金集めは、リクルート疑惑さなかの８９年に自殺した同氏の元秘書、...
4,また、リクルート事件さなかの八九年四月にも、退陣を表明したばかりの竹下登首相（当時）の元秘書...
5,八九年のリクルート事件で、竹下元首相の青木伊平元秘書。


### ELMoで文書ベクトルを算出

In [30]:
register_documents_words_vec = predict_word_vec(elmo_model, register_documents[TEXT_COLUMN].values)
register_documents_vec = calc_doc_vec(register_documents_words_vec)
print("文書ベクトルの次元数確認{shape}".format(shape=register_documents_vec.shape))

2020-04-04 15:31:02,222 INFO: 138 batches, avg len: 70.3
2020-04-04 15:31:10,887 INFO: Finished 1000 sentences.
2020-04-04 15:31:19,347 INFO: Finished 2000 sentences.
2020-04-04 15:31:29,570 INFO: Finished 3000 sentences.
2020-04-04 15:31:39,141 INFO: Finished 4000 sentences.


実行時間:0:00:43.874961
実行時間:0:00:00.201161
文書ベクトルの次元数確認(4125, 1024)


### Indexを登録する特徴量空間の定義
検索アルゴリズムと必要な初期値はこちら参照<br>
https://github.com/facebookresearch/faiss/wiki/Faiss-indexes

In [0]:
# 登録するデータ1件分の次元数
vector_dim = register_documents_vec.shape[1]
# 分割する領域の数
voronoi_num = 200
# 検索時に含める近傍の領域の数
voronoi_near = 20

#### 高速な近似アルゴリズムIndeIVFFlatを使用
https://github.com/facebookresearch/faiss/wiki/Faster-search

In [0]:
quantizer = faiss.IndexFlatL2(vector_dim)
index = faiss.IndexIVFFlat(quantizer, vector_dim, voronoi_num)

### Indexへの追加（+ボロノイ領域の学習）

In [0]:
assert not index.is_trained
index.train(register_documents_vec)
assert index.is_trained

In [34]:
# idの付与
ids = np.arange(start=1, stop=register_documents_vec.shape[0]+1)
print("idsの長さ{size}".format(size=len(ids)))
ids[:5]

idsの長さ4125


array([1, 2, 3, 4, 5])

In [0]:
index.add_with_ids(register_documents_vec, ids) 

#### 結果の保存

In [0]:
EXPORT_DIR = "./exports"
now = datetime.datetime.now().strftime('%Y%m%d%H%M%S')

In [0]:
faiss.write_index(index, EXPORT_DIR + "/features_" + now + ".index")

## 検索エンジンにクエリを投げてみる

#### 質問文の前処理

In [0]:
# 質問文
queries = ["最近傍アルゴリズムの練習にpythonで、類似文章検索エンジンを実装してみました", "野球で強い選手は誰なのか分からない", "コンピュータ", "上野動物園にパンダを見に行きたいと思うが、近くの御徒町が怖い", "野球のルールが分からない"]

In [39]:
# ELMoで単語ベクトルに変換
queries_words_vec = predict_word_vec(elmo_model, queries)
# 文書ベクトルに変換
queries_vec = calc_doc_vec(queries_words_vec)
print("入力の次元{shape}".format(shape=queries_vec.shape))

2020-04-04 15:31:40,767 INFO: 1 batches, avg len: 23.0


実行時間:0:00:00.123970
実行時間:0:00:00.000453
入力の次元(5, 1024)


#### インデックスから類似文を検索する

In [0]:
@time_watch
def search_doc(queries_vec, search_doc_num, index):
    """
    類似文章の検索

    Parameters
    -----
    queries_vec : ndarray
    search_doc_num : int
        何件分まで検索してくるか
    index : index_obj
        faissのindexオブジェクト

    Returns
    -----
    D : D_obj
    I : I_obj

    Notes
    -----
    WARNING : queries_vecは1質問文でないときの戻り値を、ドキュメントで確認中
    """
    d, i = index.search(queries_vec, search_doc_num)
    return d, i

In [41]:
search_doc_num = 10

results_distances, results_ids = search_doc(queries_vec, search_doc_num, index)

実行時間:0:00:00.000978


In [42]:
# 結果の描画
for i, (result_distances, result_ids) in enumerate(zip(results_distances, results_ids)):
    
    print("{i}番目の検索クエリ:{query}".format(i=i+1, query=queries[i]))
    print("-----検索結果-----")

    for j, (result_distance, result_id) in enumerate(zip(result_distances, result_ids)):
        if result_id != -1:
            print("第{j}位\t文書id:{id}\tテキスト:{text}\t距離{distance}"
            .format(j=j+1, id=result_id, text=register_documents[TEXT_COLUMN][result_id], distance=result_distance))
    
    print("-----{i}番目の結果ここまで-----".format(i=i+1))

1番目の検索クエリ:最近傍アルゴリズムの練習にpythonで、類似文章検索エンジンを実装してみました
-----検索結果-----
第1位	文書id:3172	テキスト:自動車大手ダイムラー・クライスラーが米クライスラー部門の経営不振にあえいでいる。//クライスラーの最大の問題は、ドル箱商品だったミニバンやスポーツ用多目的車（ＳＵＶ）が日本や欧州のライバルメーカーの攻勢で、深刻な販売不振に陥っていることだ。	距離7.2824811935424805
第2位	文書id:3415	テキスト:昨年七月、パリ・ドゴール空港近くに墜落したエールフランスの超音速旅客機コンコルドの事故で、英日曜紙オブザーバーは十三日、//「真の原因」はエールフランスの整備不良にあったと論じている。	距離8.013826370239258
第3位	文書id:2125	テキスト:Ｊ・Ｋ・ローリング著　ハリー・ポッターとアズカバンの囚人//皆さんは「ハリポタ」をご存知かな？//世界中で爆発的に売れ、出版史上空前の売り上げを記録しつつあるイギリスのファンタジー小説シリーズのこと。	距離8.32452392578125
第4位	文書id:2327	テキスト:アフガニスタンを実効支配するイスラム原理主義組織タリバンが、バーミヤンの石仏をほぼ完全に破壊したことが１２日、国連によって確認された。	距離8.443242073059082
第5位	文書id:2549	テキスト:宮崎市の大型リゾート施設・シーガイアを米投資会社のリップルウッド・ホールディングスが引き継ぐことが正式決定した二十九日、県や市、労組の関係者は安どの表情を見せた。//◇シーガイアの歩み//リップルウッド・ホールディングスがスポンサー基本契約を締結「再建に向けて大きな前進」。	距離8.561403274536133
第6位	文書id:723	テキスト:インサイトはガソリンエンジンに電気モーターを組み合わせて燃費向上を図る「ハイブリッドシステム」と軽量のアルミボディーを採用し、ガソリン一リットル当たり世界最高の三十五キロ・メートルを走ることができる。二人乗りの乗用車で、	距離8.665226936340332
第7位	文書id:3214	テキスト:アメリカ東岸のボストン空港に、トランクを両手に抱えて到着した野依良治さん（６３）の姿は、雪の吹きすさ